# Intro & Metadata

This notebook introduces the approach of the current repository, and explores the metadata files.

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import yaml

In [4]:
# Import local code module
import sys
sys.path.append('..')
from conv import process_logfile
from conv.parser import parse_lines_logfile
from conv.io import *

## Metadata Files

Metadata for the task is organized into a series of config files, each representing a facet of the study / task. 

In order to process an individual subject, the configs can be loaded together and session specific information can be added. 

This combined metadata file is then used to create the output data file. 

In [5]:
# Get a list of metadata files
files = get_files('../metadata', 'yaml')

In [6]:
# Check the list of configuration files
files

['subject_info.yaml',
 'electrode_info.yaml',
 'study_info.yaml',
 'device_info.yaml',
 'task_info.yaml',
 'events_info.yaml']

In [7]:
# Load all the files together to create an overall metadata object
metadata = load_configs(files, '../metadata')

In [8]:
# Check the metadata object
metadata

{'subject': {'age': 'XX',
  'sex': 'XX',
  'description': 'XX',
  'species': 'human'},
 'electrode': {'name': 'XX',
  'description': 'Behnke Fried/Micro Inner Wire Bundle.',
  'location': 'XX',
  'impedence': 'XX',
  'filtering': 'XX',
  'reference': 'XX',
  'position': 'XX, XX, XX'},
 'study': {'identifier': 'XX',
  'session_id': 'XX',
  'session_start_time': '1111/11/11',
  'session_description': 'XX',
  'experiment_description': 'XX',
  'experimenter': 'XX',
  'lab': 'Electrophysiology, Memory, and Navigation Laboratory',
  'institution': 'Columbia University',
  'data_collection': 'XX'},
 'device': {'name': 'Microwire Electrodes',
  'description': 'Behnke Fried Micro Inner Wire Bundle',
  'manufacturer': 'Ad-Tech Medical'},
 'task': {'name': 'T3',
  'description': 'XX',
  'events_description': 'XX',
  'task_run_version': 'XX',
  'revision_date': 'XX',
  'run_platform': 'Unity'},
 'events': None}

The above metadata file includes all the metadata fields

Where available, information is prefilled with default information for the current task. 

For fields not filled in (marked with `XX`), the metadata needs to be entered. 

In [10]:
# Save out the collected metadata to a subject-level file
save_config(metadata, 'example_files/example_metadata')

In [11]:
# Reload the collected metadata file, and check an example attribute
metadata_new = load_config('example_files/example_metadata.yaml')
metadata_new['study']['lab']

'Electrophysiology, Memory, and Navigation Laboratory'

## Logfile Processing

The task logfile, which is a structured txt file, needs parsing and organizing in order to create the new data files. 

In [12]:
# Define base data folder
DATA_FOLDER = Path('...')

# Define location of the logfile
logfile_path = DATA_FOLDER / 'behavioral' / 'logfile.txt'

### Process logfile

The logfile processing extracts required information from the logfile into a `Task` object.

In [ ]:
# Parse the log file
task = parse_lines_logfile(logfile_path)

In [ ]:
# Check the task object
task

### Process Functions

This `Task` object can then be passed into subsequent function(s) to process the information.

These process functions include:
- `process_time_info`
- `process_task_info`
- `process_position_info`
- `process_location_info`
- `process_error_info`

Each of these functions can be updated independently if some task-related information needs to be updated. 

In [13]:
# Import the process functions
from conv.process import *